In [44]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers

### Simulation (in Matrix-Representation)

Agents (NNs) are given bird persepective images of his square shaped world.
Their Action Space contains the Options staying, or moving (right, left, up, down).
Mit Hilfe von Linarer Algrebrah wird, dem output des Agents entsprechend, ein Foto ihrer Welt im nächsten Simulations-Schritt berechnet.
Nach S Simulations-Schritten wird die Position des Agents der Fitness Function übergeben.

The dimentions of the world:

In [45]:
L, W = 4, 4

Agent:

In [95]:
def get_agent_model(L, W):

    A = layers.Input(shape=(L,W,2))
    h = layers.Flatten()(A)
    h = layers.Dense(
        4, 
        activation='sigmoid',
        bias_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
    )(h)
    # h = tf.linalg.normalize(h)
    # h = layers.LayerNormalization()(h)  
    h = tf.math.divide(
        tf.math.subtract(
            h, 
            tf.reduce_min(h)
        ), 
        tf.math.subtract(
            tf.reduce_max(h), 
            tf.reduce_min(h)
        )
    )
    motion = tf.math.round(h, name='agents_motion')

    return tf.keras.Model(A, motion, name='agent_nn')

In [104]:
from turtle import shape


def get_enviroment_model(A, M, K):

    # New Position Vektor K acording to motion M

    # M_1 = tf.math.scalar_mul(
    #         scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[1,0,0,0]], dtype=tf.float32))),
    #         x=tf.constant([[-1,0]], dtype=tf.float32),
    #         name='coord. step up'
    #     )

    # M_2 = tf.math.scalar_mul(
    #         scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,1,0,0]], dtype=tf.float32))),
    #         x=tf.constant([[1,0]], dtype=tf.float32),
    #         name='coord. step down'
    #     )

    # M_3 = tf.math.scalar_mul(
    #         scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,1,0]], dtype=tf.float32))),
    #         x=tf.constant([[0,-1]], dtype=tf.float32),
    #         name='coord. step left'
    #     )

    # M_4 = tf.math.scalar_mul(
    #         scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,0,1]], dtype=tf.float32))),
    #         x=tf.constant([[0,1]], dtype=tf.float32),
    #         name='coord. step right'
    #     )

    # M_12 = tf.math.add(M_1, M_2)
    # M_34 = tf.math.add(M_3, M_4)
    # M_all = tf.math.add(M_12, M_34)
    # K_new = tf.math.add(K, M_all)
    
    K_new = layers.Dense(units=2)(K)



    # # New image B according to motion M

    # kernel = np.zeros(shape=(3,3,2,2))
    # kernel[1,1,1,1] = 1                     # The convolution on the target dimention in the enviroment's image is the identity


    # '''Up'''
    # conv_layer_up = layers.Conv2D(filters=2, kernel_size=(3,3), padding='same', kernel_initializer=tf.constant_initializer(1.))
    # conv_layer_up_output = conv_layer_up(A)
    
    # kernel_up = kernel.copy()
    # kernel_up[2,1,0,0] = 1 
    # conv_layer_up.set_weights([tf.constant(kernel_up, dtype=tf.float32), tf.constant(0, shape=(2))])                                  

    # A_1 = tf.math.scalar_mul(
    #     scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[1,0,0,0]], dtype=tf.float32))),
    #     x=conv_layer_up_output, 
    #     name='image_step_up_multi'
    # )
    # A_1 = tf.math.add(A_1, tf.math.scalar_mul(scalar=1 - tf.reduce_sum(tf.math.multiply(M, tf.constant([[1,0,0,0]], dtype=tf.float32))), x=A), name='image_step_up_add')

    # '''Down'''
    # conv_layer_down = layers.Conv2D(filters=2, kernel_size=(3,3), padding='same', kernel_initializer=tf.constant_initializer(1.))
    # conv_layer_down_output = conv_layer_down(A_1)
    
    # kernel_down = kernel.copy()
    # kernel_down[0,1,0,0] = 1 
    # conv_layer_down.set_weights([tf.constant(kernel_down, dtype=tf.float32), tf.constant(0, shape=(2))])

    # A_2 = tf.math.scalar_mul(
    #     scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,1,0,0]], dtype=tf.float32))),
    #     x=conv_layer_down_output
    # )
    # A_2 = tf.math.add(A_2, tf.math.scalar_mul(scalar=1 - tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,1,0,0]], dtype=tf.float32))), x=A_1))

    # '''Left'''
    # conv_layer_left = layers.Conv2D(filters=2, kernel_size=(3,3), padding='same', kernel_initializer=tf.constant_initializer(1.))
    # conv_layer_left_output = conv_layer_left(A_2)
    
    # kernel_left = kernel.copy()
    # kernel_left[1,2,0,0] = 1 
    # conv_layer_left.set_weights([tf.constant(kernel_left, dtype=tf.float32), tf.constant(0, shape=(2))])

    # A_3 = tf.math.scalar_mul(
    #     scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,1,0]], dtype=tf.float32))),
    #     x=conv_layer_left_output
    # )
    # A_3 = tf.math.add(A_3, tf.math.scalar_mul(scalar=1 - tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,1,0]], dtype=tf.float32))), x=A_2))

    # '''Right'''
    # conv_layer_right = layers.Conv2D(filters=2, kernel_size=(3,3), padding='same', kernel_initializer=tf.constant_initializer(1.))
    # conv_layer_right_output = conv_layer_right(A_3)
    
    # kernel_right = kernel.copy()
    # kernel_right[1,0,0,0] = 1 
    # conv_layer_right.set_weights([tf.constant(kernel_right, dtype=tf.float32), tf.constant(0, shape=(2))])

    # A_4 = tf.math.scalar_mul(
    #     scalar=tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,0,1]], dtype=tf.float32))),
    #     x=conv_layer_right_output
    # )
    # A_new = tf.math.add(A_4, tf.math.scalar_mul(scalar=1 - tf.reduce_sum(tf.math.multiply(M, tf.constant([[0,0,0,1]], dtype=tf.float32))), x=A_3))
   
    A_new = layers.Flatten()(A)
    A_new = layers.concatenate([A_new, M])
    A_new = layers.Dense(units=L*W*2)(A_new)
    A_new = layers.Reshape(target_shape=(L, W, 2))(A_new)
    

    # A_new = A + tf.scalar_mul(layers.Dense(1)(M), np.ones(shape=(L,W,2), dtype=np.float32))  
    

    # return tf.keras.Model((A, M, K), (A_new, K_new), name='Enviroment')
    return tf.keras.Model((A, M, K), (K_new), name='Enviroment')

### Fitness Function (in Matrix-Representation)

Subtration der Target-Position von den x,y Koordianten

In [75]:
pop_num = 4
A = layers.Input(shape=(2, pop_num))
A_sqr = tf.math.multiply(A, A)

A_split_x = tf.split(A_sqr, 2, axis=1, name='split')[0]
A_split_y = tf.split(A_sqr, 2, axis=1, name='split')[1]

A_sum = tf.math.add(A_split_x, A_split_y)
A_norm = tf.linalg.normalize(A_sum)

# A_slice_a = tf.slice(A_sqr, [0,0,0], [1,2,1], name=None) 
# A_slice_b = tf.slice(A_sqr, [0,1,0], [1,2,1], name=None) 
# A_ave = layers.AveragePooling2D(pool_size=(2))(A)


In [82]:
fitness_function = tf.keras.Model(A, A_norm)

print(fitness_function.predict(tf.constant([[[1,2,3,4],[5,6,7,8]]])))

# Das Netzwert berechnet die Abstandsnorm der x,y Koordinaten zum ursprung.
# Für die Nutzung als Fitness Function ist der Output des Models normiert.

(array([[[0.23695184, 0.3645413 , 0.5285849 , 0.7290826 ]]], dtype=float32), array([[[109.72694]]], dtype=float32))


### Reproduktion (in Matrix-Representation - Tensorflow)

#### Option 1:

In [ ]:
for i in range(generations):
    fitness_function(final)

    final_coordinates = experience(population)               # Array contains final coordinates of all agents in the population (2 x P)
    fitness_function(final_coordinates)

    # The 50'th best (closest) agents get to repopulate and cross mutate
    # Mutation: Addition of random tensors and agent's model's weights
    # Crossmutation: Split tensors of model's weights and concat slices of two different agents

What if we could implement the selection and reproduction process as an optimizer?

Anstonsten könnte man den Selectionsprozess wie folgt implementieren:

In [ ]:
list_of_50_best_agents = None
list_of_50_best_agents = .shuffel(list_of_50_best_agents)


old_agent = list_of_50_best_agents[-1]
for agent in list_of_50_best_agents[:-2]:
    # split agent.model.weights and concatanate with old_agent.
    # Create both combinations.
    
    old_agent.model = None
    agent.model = None

    old_agent = agent

new_agents = [list_of_50_best_agents] * 2

#### Option 2:

Geringere Anzahl an Generations dafür aber Training der Agents mit tf.keras.Model.fit()

In [100]:
def get_experience_model_original(agent_model, L, W, simulation_steps=10):
    
    

    A = layers.Input(shape=(L,W,2), name='image_enviroment')
    K = layers.Input(shape=2, name='agent_position')

    enviroment = get_enviroment_model(A, agent_model.output, K)
    enviroment.trainable = False

    agent_motion = agent_model(A)
    (A_new, K_new) = enviroment((A, agent_motion, K))

    for experience in range(simulation_steps - 1):
        agent_motion = agent_model(A_new)
        (A_new, K_new) = enviroment((A_new, agent_motion, K_new))

    model_experience = tf.keras.Model((A, K), (K_new), name='one_experience')

    # T = layers.Input(shape=2, name='target_position')
    # model_experience.add_loss(tf.keras.losses.mse(K_new, T))
    
    return model_experience

In [113]:
def get_experience_model(agent_model, L, W, simulation_steps=10):
    
    A = layers.Input(shape=(L,W,2), name='image_enviroment')
    K = layers.Input(shape=2, name='agent_position')
    M = layers.Input(shape=4, name='agents_motion')

    # enviroment = get_enviroment_model(A, M, K)
    # enviroment.trainable = False

    dense1 = layers.Dense(units=L*W*2)
    dense2 = layers.Dense(units=2)

    M = agent_model(A)
    A_new = layers.Flatten()(A)
    A_new = layers.Concatenate()([A_new, M])
    A_new = dense1(A_new)
    A_new = layers.Reshape(target_shape=(L, W, 2))(A_new)
    K_new = dense2(K)

    # K_new = enviroment((A, M, K))

    # for experience in range(simulation_steps - 2):
    #     M = agent_model(A_new)
    #     A_new = layers.Flatten()(A_new)
    #     A_new = layers.Concatenate()([A_new, M])
    #     A_new = dense1(A_new)
    #     A_new = layers.Reshape(target_shape=(L, W, 2))(A_new)
    #     K_new = dense2(K_new)
        
    #     # (A_new, K_new) = enviroment((A_new, agent_motion, K_new))

    model_experience = tf.keras.Model((A, K), (K_new), name='one_experience')
    
    return model_experience

##### Generate trainng and validation data:

In [10]:
def get_enviroment_image(agent_pos, target_pos, raw_image):
    raw_image[agent_pos[0], agent_pos[1], 0] = 1          # Place the agent in the image of the enviroment
    raw_image[target_pos[0], target_pos[1], 1] = 1        # Place the target in the image of the enviroment
    return raw_image

In [11]:
data_size = 1000
validation_split=0.7

train_size = int(data_size * validation_split)
val_size = data_size - train_size

agent_positions = np.array([np.random.randint(size=train_size, low=0, high=L),
                        np.random.randint(size=train_size, low=0, high=W)]).reshape((train_size, 2))
val_agent_positions = np.array([np.random.randint(size=val_size, low=0, high=L),
                        np.random.randint(size=val_size, low=0, high=W)]).reshape((val_size, 2))                            

target_positions = np.array([np.random.randint(size=train_size, low=0, high=L),
                        np.random.randint(size=train_size, low=0, high=W)]).reshape((train_size, 2))
val_target_positions = np.array([np.random.randint(size=val_size, low=0, high=L),
                        np.random.randint(size=val_size, low=0, high=W)]).reshape((val_size, 2))

raw_image = np.zeros(shape=(L,W,2))
train_images = np.array([get_enviroment_image(agent_positions[i], target_positions[i], raw_image) for i in range(train_size)])
val_images = np.array([get_enviroment_image(val_agent_positions[i], val_target_positions[i], raw_image) for i in range(val_size)])

##### Generator Approach: (Failed)

In [50]:
# def get_enviroment_image(agent_pos, target_pos, raw_image):
#     raw_image[agent_pos[0], agent_pos[1], 0] = 1          # Place the agent in the image of the enviroment
#     raw_image[target_pos[0], target_pos[1], 1] = 1        # Place the target in the image of the enviroment
#     return tf.constant([raw_image])

In [134]:
# def get_dataset_generator(data_size, raw_image):
#     for i in range(data_size):
#         agent_pos = [np.random.randint(low=0, high=L),
#                     np.random.randint(low=0, high=W)]
#         target_pos = [np.random.randint(low=0, high=L),
#                     np.random.randint(low=0, high=W)]

#         image = get_enviroment_image(agent_pos=agent_pos, target_pos=target_pos,
#                                         raw_image=raw_image.copy())
#         # yield (tf.Tensor(image), tf.Tensor(agent_pos), tf.Tensor(target_pos))
#         yield ((image, tf.constant(agent_pos, dtype=tf.int32), tf.constant(target_pos, dtype=tf.int32)), (tf.constant(target_pos, dtype=tf.int32)))

In [135]:
# image = np.zeros(shape=(L,W,2))
# dataset_generator = get_dataset_generator(1000, image)

##### tf.data.Dataset approach: (Failed)

In [108]:
# dataset_generator = get_dataset_generator(1000, image)
from unicodedata import name


# dataset = tf.data.Dataset.from_generator(get_dataset_generator,
#         output_signature=(
#             tf.TensorSpec(shape=(L,W,2)),
#             tf.TensorSpec(shape=(2)),
#             tf.TensorSpec(shape=(2))
#         )
#     )

##### Training:

In [114]:
agent_nn = get_agent_model(L, W)
model_experience = get_experience_model(agent_nn, L, W, simulation_steps=5)


model_experience.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
                        loss=['mse', 'mse'],
                        metrics=['mse', 'acc'])

model_experience.fit(x=(train_images, agent_positions), y=target_positions, epochs=10, batch_size=64)

# model_experience.fit(x=XXX Generator Input XXX, epochs=5)
# model_experience.fit(x=XXX tf.data.Dataset Input XXX epochs=5)

Epoch 1/10
11/11 [==============================] - 0s 0s/step - loss: 2.3555 - mse: 2.3555 - acc: 0.4900
Epoch 2/10
11/11 [==============================] - 0s 800us/step - loss: 1.5527 - mse: 1.5527 - acc: 0.4429
Epoch 3/10
11/11 [==============================] - 0s 800us/step - loss: 1.4341 - mse: 1.4341 - acc: 0.4414
Epoch 4/10
11/11 [==============================] - 0s 800us/step - loss: 1.3619 - mse: 1.3619 - acc: 0.5100
Epoch 5/10
11/11 [==============================] - 0s 1ms/step - loss: 1.3183 - mse: 1.3183 - acc: 0.4629
Epoch 6/10
11/11 [==============================] - 0s 800us/step - loss: 1.3038 - mse: 1.3038 - acc: 0.4300
Epoch 7/10
11/11 [==============================] - 0s 810us/step - loss: 1.2815 - mse: 1.2815 - acc: 0.5157
Epoch 8/10
11/11 [==============================] - 0s 2ms/step - loss: 1.2632 - mse: 1.2632 - acc: 0.4586
Epoch 9/10
11/11 [==============================] - 0s 800us/step - loss: 1.2528 - mse: 1.2528 - acc: 0.5014
Epoch 10/10
11/11 [=======

Die Funktionen get_agent_model und get_enviroment_model sollen sich gegenseitig Tensoren übergeben und so mehrfach hintereinander sequenziert ein multiples durchlaufen von action-feedback loops simulieren.
Keine Probleme bringt die Einbindung vom Agent's Model in ein größeres tf.keras.Model, welches optimiert werden soll (agents weights sind die einzigen trainierbaren).
Wird allerdings die enviroment mit eingebunden so wird ein Fehler ('fehlender gradient') geworfen.


Führt zu erfolgreichem Training:
*   agent_model gibt M an sequential von dense, reshape und flatten weiter (1 Runde) - tf.keras.Model((A, K), (K_new), name='one_experience')

Fürt zum Durchlaufen der Trainingsepochen aber loss/ mse (nicht acc) sind mitunter gleich nan
*   agent_model gibt M an sequential von dense, reshape und flatten weiter (N > 1 Runde) - tf.keras.Model((A, K), (K_new), name='one_experience')

Wann auch immer ich ein weiteres tf.keras.Model einbinden möchte wird der fehler mit den fehlenden gradienten geschmissen!

("I know this is closed, but I thought I might mention for the sake of Googler's: most non-linear operations (such as tf.cast) do not provide gradients. "
https://github.com/tensorflow/tensorflow/issues/1511)

In [ ]:
# Create Random Training Data:
position = np.array([[1,1]])
target_position = np.array([[3,3]])

image = np.zeros(shape=(L,W,2))
image[position[:,0], position[:,1], 0] = 1                      # Place the agent in the image of the enviroment
image[target_position[:,0], target_position[:,1], 1] = 1        # Place the target in the image of the enviroment
image = tf.constant([image])


# Create Model in order to training an agent:
agent_nn = get_agent_model(L, W)
model_experience = get_experience_model(agent_nn, L, W, 10)
# model_experience.predict((image, position, target_position))

# print('Before motion:')
# print(np.swapaxes(np.squeeze(image), 0, 2))

# print('Position:', position)

print('\nAfter motion:')
# print(np.squeeze(model_life.predict((image, position))[0]))
# print(np.swapaxes(np.squeeze(model_experience.predict((image, position, target_position))[0]), 0, 2))
# print('Position:', model_experience.predict((image, position, target_position))[1])


print(model_experience.predict((image, position)))

In [23]:
def life(agent_model, days_in_life, data_size=1000, validation_split=0.7):
    
    ''' Generate training and validation data: '''
    train_size = int(data_size * validation_split)
    val_size = data_size - train_size

    agent_positions = np.array([np.random.randint(size=train_size, low=0, high=L),
                            np.random.randint(size=train_size, low=0, high=W)]).reshape((train_size, 2))
    val_agent_positions = np.array([np.random.randint(size=val_size, low=0, high=L),
                            np.random.randint(size=val_size, low=0, high=W)]).reshape((val_size, 2))                            

    target_positions = np.array([np.random.randint(size=train_size, low=0, high=L),
                            np.random.randint(size=train_size, low=0, high=W)]).reshape((train_size, 2))
    val_target_positions = np.array([np.random.randint(size=val_size, low=0, high=L),
                            np.random.randint(size=val_size, low=0, high=W)]).reshape((val_size, 2))                            

    image = np.zeros(shape=(L,W,2))
    def get_enviroment_image(agent_pos, target_pos, raw_image):
        raw_image[agent_pos[0], agent_pos[1], 0] = 1          # Place the agent in the image of the enviroment
        raw_image[target_pos[0], target_pos[1], 1] = 1        # Place the target in the image of the enviroment
        return tf.constant([raw_image])

    images = [get_enviroment_image(agent_pos=random_agent_positions[i],
        target_pos=target_pos,
        raw_image=image.copy()) for i, target_pos in enumerate(random_target_positions)]
    val_images = [get_enviroment_image(
        agent_pos=val_random_agent_positions[i], 
        target_pos=val_target_pos, 
        raw_image=image.copy()) for i, val_target_pos in enumerate(val_random_target_positions)]


    ''' Create, compile, and train model: '''
    model_experience = get_experience_model(agent_model)
    
    model_experience.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1))

    model_experience.fit(random_target_positions, random_target_positions, epochs=days_in_life)
    
    # return model_life.evaluate(test_random_target_positions)[index_of_the_accuracy]

In [186]:
# life(agent_nn, days_in_life=5, training_data_size=1000, validation_split=0.7)

In [8]:
# for i in range(generations):
#     # Train for T epochs corresponding to the experience of one life-time
    
#     for i, agent in population.agents:
#         fitness_scores[i] = life(agent, days_in_life=days_in_life)


#     # The 50'th best (closest) agents get to repopulate and cross mutate
#     # Crossmutation: Split tensors of model's weights and concat slices of two different agents